# Rating Product & Sorting Reviews in Amazon

One of the most important problems in e-commerce is the correct calculation of the points given to the products after sales. The solution to this problem means providing greater customer satisfaction for the e-commerce site, prominence of the product for the sellers and a seamless shopping experience for the buyers. Another problem is the correct ordering of the comments given to the products. Since misleading comments will directly affect the sale of the product, it will cause both financial loss and loss of customers. In the solution of these 2 basic problems, e-commerce site and sellers will increase their sales, while customers will complete their purchasing journey without any problems.

This dataset, which includes Amazon product data, includes product categories and various metadata. Best in electronics
The product with more reviews has user ratings and reviews.

*reviewerID* : User ID<br>
*asin* : Product ID<br>
*reviewerName* : Username<br>
*helpful* : Useful rating rating<br>
*reviewText* : Review<br>
*overall* : Product rating<br>
*summary* : Evaluation summary<br>
*unixReviewTime* : Evaluation time<br>
*reviewTime* : Reviewtime Raw<br>
*day_diff* : Number of days since evaluation<br>
*helpful_yes* : The number of times the evaluation was found helpful<br>
*total_vote* : The number of votes given to the rating<br>

In [1]:
# libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import math
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as st

pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)

In [2]:
data = pd.read_csv("datas/amazon_review.csv")
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.0,4.587589e+00,9.968451e-01,1.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
unixReviewTime,4915.0,1.379465e+09,1.581857e+07,1.339200e+09,1.365898e+09,1.381277e+09,1.392163e+09,1.406074e+09
day_diff,4915.0,4.373670e+02,2.094399e+02,1.000000e+00,2.810000e+02,4.310000e+02,6.010000e+02,1.064000e+03
helpful_yes,4915.0,1.311089e+00,4.161916e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.952000e+03
total_vote,4915.0,1.521465e+00,4.412309e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.020000e+03


In [5]:
data.isnull().sum()

reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

Out of a total of 4915 entries from the attributes, only two attributes seem to be missing 1 value. Since the ratio of missing values to data is not very high, this can be ignored.

In the shared data set, users gave points and comments to a product. Our aim in this task is to evaluate the scores given by weighting them by date. It is necessary to compare the first average score with the weighted score according to the date to be obtained.

In [6]:
# Calculating the average score of the product
data["overall"].mean()

4.587589013224822

#### Calculation of weighted average score by date

In [7]:
data["reviewTime"] = pd.to_datetime(data["reviewTime"])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reviewerID      4915 non-null   object        
 1   asin            4915 non-null   object        
 2   reviewerName    4914 non-null   object        
 3   helpful         4915 non-null   object        
 4   reviewText      4914 non-null   object        
 5   overall         4915 non-null   float64       
 6   summary         4915 non-null   object        
 7   unixReviewTime  4915 non-null   int64         
 8   reviewTime      4915 non-null   datetime64[ns]
 9   day_diff        4915 non-null   int64         
 10  helpful_yes     4915 non-null   int64         
 11  total_vote      4915 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 460.9+ KB


In [8]:
current_date = data["reviewTime"].max()  # max date-time
current_date

Timestamp('2014-12-07 00:00:00')

In *weighted scoring*, a few steps will be followed to compare and interpret the average of each time period. *Average Rating* will be calculated according to current comments and compared with the existing average rating. The *reviewTime* variable will be introduced as a date variable. It will set the max value of *reviewTime* to *current_date*. A new variable will be created by expressing the difference of each score-comment date and *current_date* in days. The variable will be divided by 4 with the quantile function and weighted according to the values from the quarters. For example, if q1 = 12, when weighting, averaging comments made less than 12 days ago and giving them higher weights.

In [9]:
data["days"] = (current_date - data["reviewTime"]).dt.days
data.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512


In [10]:
# the oldest review is from 1063 days ago
data["days"].max()

1063

In [11]:
data["days"].quantile([.1, .25, .5, .75])

0.10    166.0
0.25    280.0
0.50    430.0
0.75    600.0
Name: days, dtype: float64

In [12]:
print(data.loc[data["days"] <= 166, "overall"].mean())
print(data.loc[(data["days"] > 166) & (data["days"] <= 280), "overall"].mean())
print(data.loc[(data["days"] > 280) & (data["days"] <= 430), "overall"].mean())
print(data.loc[(data["days"] > 430) & (data["days"] < 600), "overall"].mean())
print(data.loc[data["days"] > 600, "overall"].mean())

4.68986083499006
4.699863574351978
4.636140637775961
4.571428571428571
4.4462540716612375


In the first review, there is an increase in the votes in the comments close to the current day compared to the previous evaluations. The reason for this may be that the seller has increased its quality and performance, or that the seller or the product has changed.

In [13]:
def time_based_weighted_average(dataframe,feature,w1,w2,w3,w4,w5):

    return dataframe.loc[dataframe[feature] <= 166, "overall"].mean() * w1/100 + \
    dataframe.loc[(dataframe[feature] > 166) & (dataframe[feature] <= 280), "overall"].mean() * w2/100 + \
    dataframe.loc[(dataframe[feature] > 280) & (dataframe[feature] <= 430), "overall"].mean() * w3/100 + \
    dataframe.loc[(dataframe[feature] > 430) & (dataframe[feature] < 600), "overall"].mean() * w4/100 + \
    dataframe.loc[dataframe[feature] > 600, "overall"].mean() * w5/100

In [14]:
time_based_weighted_average(data,"days",30,25,23,12,10)

4.641433326511034

The reason for giving higher weight to days less than 160 is because up-to-date comments are always more important and useful. More than 280 days is a comment almost years ago. During this process, the seller may have changed, the product quality may have changed. Therefore, that comment may not be useful and may not reflect the current truth.

### Determining 20 reviews to be displayed on the product detail page for the product

*total_vote* is the total number of up-downs given to a comment. There is no helpful_no variable in the data set, it must be generated over existing variables. For this reason, the number of useful votes (*helpful_yes*) should be subtracted from the total number of votes (*total_vote*) and the number of votes that are not found useful (*helpful_no*) should be found.

In [15]:
data["helpful_no"] = data["total_vote"] - data["helpful_yes"]
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,381,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,512,0


### Calculating score_pos_neg_diff, score_average_rating and wilson_lower_bound scores and adding them to the data

In order to calculate score_pos_neg_diff, score_average_rating and wilson_lower_bound scores, score_pos_neg_diff, score_average_rating and wilson_lower_bound functions defined. Then scores are created according to score_pos_neg_diff and score_average_rating. Finally, scores are created according to wilson_lower_bound.

In [16]:
def score_pos_neg_diff(up_rates,down_rates):
    
    return (up_rates - down_rates)

In [17]:
def score_average_rating(up_rates, down_rates):
    
    if up_rates + down_rates == 0:
        return 0
    
    return up_rates / (up_rates + down_rates)

In [18]:
def wilson_lower_bound(up_rates, down_rates, confidence=0.95):
    
    n = up_rates + down_rates
    
    if n == 0:
        return 0
    
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up_rates / n
    
    return (phat + z * z / (2*n) - z * math.sqrt((phat * (1- phat) + z * z / (4*n)) / n)) / (1 + z * z/n)

In [19]:
data["score_pos_neg_diff"] = data.apply(lambda x: score_pos_neg_diff(x["helpful_yes"],x["helpful_no"]), axis=1)

data["score_average_rating"] = data.apply(lambda x: score_average_rating(x["helpful_yes"],x["helpful_no"]), axis=1)

data["wilson_lower_bound"] = data.apply(lambda x: wilson_lower_bound(x["helpful_yes"],x["helpful_no"]), axis=1)

In [21]:
data.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701,68,1884,0.966337,0.957544
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802,77,1351,0.948837,0.936519
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578,126,1442,0.925620,0.912139
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032,73,349,0.852525,0.818577
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,157,4,41,0.918367,0.808109
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0,I own it,1393545600,2014-02-28,283,60,68,282,8,52,0.882353,0.784651
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,776,27,85,0.805755,0.732136
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.0,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,648,3,19,0.880000,0.700442
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0,Awesome Card!,1339200000,2012-09-06,823,51,65,822,14,37,0.784615,0.670334
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,806,27,55,0.752294,0.663595
